# L-BFGS and Vector-free L-BFGS demonstration

In [183]:
import numpy as np
import lbfgs
import utils
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import check_grad

## Check of the convergence 

We check that the algorithms implemented for L-BFGS and Vector-free L-BFGS (VL-BFGS) converges correctly by comparing them to the corresponding algorithm of scipy.

#### Simulate data

In [185]:
# Parameters
d = 50
n = 1000
idx = np.arange(d)
lbda = 1. / n ** (0.5)

# Ground truth coefficients of the model
coefs = (-1) ** idx * np.exp(-idx / 10.)

In [188]:
_A, _b = utils.simulate_data(coefs, n, std=1., corr=0.8, log_reg=False)

#### Check of the gradient computation

In [189]:
check_grad(utils.loss_logreg, utils.grad_logreg, np.random.randn(d), _A, _b, lbda)

3.185094006582731e-07

#### Optimization for linear regression

In [190]:
f = utils.loss_linreg
f_grad = utils.grad_linreg

x_init = np.zeros(d)

In [191]:
x_min_scipy, f_min_scipy, _ = fmin_l_bfgs_b(f, x_init, f_grad, args=(_A, _b, lbda), pgtol=1e-30, factr=1e-30, maxiter=100)

lbfgs_optimizer = lbfgs.LBFGS(f, f_grad, m=10, vector_free=False, max_iter=100)
x_min_lbfgs, f_min_lbfgs, time_lbfgs  = lbfgs_optimizer.fit(x_init, _A, _b, lbda)

vlbfgs_optimizer = lbfgs.LBFGS(f, f_grad, m=10, vector_free=True, max_iter=100)
x_min_vlbfgs, f_min_vlbfgs, time_vlbfgs = vlbfgs_optimizer.fit(x_init, _A, _b, lbda)

In [194]:
print("----- Results: -----")
print("Scipy minimum: ", round(f_min_scipy, 5))
print("L-BFGS minimum: ", round(f_min_lbfgs[-1], 5))
print("VL-BFGS minimum: ", round(f_min_vlbfgs[-1], 5))

----- Results: -----
Scipy minimum:  0.56451
L-BFGS minimum:  0.56451
VL-BFGS minimum:  0.56451


## Comparison of efficiency